In [ ]:
import speech_recognition as sr
import numpy as np
import pandas as pd
import time
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from joblib import dump, load

# 数据集路径
DATA_PATH = 'wifi_localization.txt'

def load_wifi_data(data_path):
    """加载Wi-Fi定位数据并进行预处理"""
    if not os.path.exists(data_path):
        print(f"Error: Data file '{data_path}' not found.")
        return None

    print(f"Loading Wi-Fi localization data from {data_path}...")
    data = pd.read_csv(data_path, sep='\t', header=None)
    data.columns = [f"Signal_{i+1}" for i in range(7)] + ["Room"]
    return data

# 加载 Wi-Fi 室内定位模型和标准化器
def load_wifi_model():
    """加载保存的Wi-Fi模型和标准化器"""
    try:
        svm_wifi_model = load('svm_wifi_localization_model.joblib')
        scaler = load('scaler.joblib')
        print("Models loaded successfully.")
    except FileNotFoundError:
        print("Model files not found. Training new models...")
        svm_wifi_model, scaler = preprocess_wifi_data()  # 如果没有模型，则训练模型
    return svm_wifi_model, scaler

# 获取实时环境数据（模拟）
def get_real_time_data():
    return {
        'Temperature': np.random.uniform(15, 30),
        'Humidity': np.random.uniform(30, 80),
        'Light_Intensity': np.random.uniform(0, 1),
        'Time_of_Day': np.random.randint(0, 24),
        'User_Home': np.random.randint(0, 2),
        'Weekday': np.random.randint(0, 2),
        'User_Preference': np.random.uniform(0, 1)
    }

# 训练并预处理 Wi-Fi 定位模型
def preprocess_wifi_data():
    data = load_wifi_data(DATA_PATH)
    if data is None:
        print("No data loaded. Exiting...")
        return None, None

    X = data.drop("Room", axis=1)
    y = data["Room"]
    
    # 拆分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 标准化数据
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # 训练SVM模型
    print("Training SVM model for Wi-Fi localization...")
    svm_wifi_model = SVC(kernel='linear', random_state=42)
    svm_wifi_model.fit(X_train_scaled, y_train)
    
    # 使用 `.joblib`格式保存模型和标准化器
    dump(svm_wifi_model, 'svm_wifi_localization_model.joblib')
    dump(scaler, 'scaler.joblib')
    
    print(f"Training accuracy: {svm_wifi_model.score(X_test_scaled, y_test):.2f}")
    return svm_wifi_model, scaler

# 语音控制系统
def listen_to_user_command():
    recognizer = sr.Recognizer()
    mic = sr.Microphone()

    with mic as source:
        print("Listening for commands...")
        recognizer.adjust_for_ambient_noise(source, duration=1)
        audio = recognizer.listen(source)

    try:
        command = recognizer.recognize_google(audio)
        print(f"Recognized command: {command}")
        return command.lower()
    except sr.UnknownValueError:
        print("Sorry, I could not understand the command.")
        return None
    except sr.RequestError as e:
        print(f"Recognition service error: {e}")
        return None

# 基于 Wi-Fi 信号预测房间
def predict_room(svm_wifi_model, scaler):
    wifi_signals = np.random.uniform(-90, -30, size=(1, 7))
    wifi_signals_df = pd.DataFrame(wifi_signals, columns=[f"Signal_{i+1}" for i in range(7)])
    wifi_signals_scaled = scaler.transform(wifi_signals_df)
    predicted_room = svm_wifi_model.predict(wifi_signals_scaled)[0]
    return predicted_room

# 基于房间定位自动控制灯光
def control_lights_based_on_room(new_room, current_room, lights):
    room_mapping = {
        1: "Bedroom",
        2: "Kitchen",
        3: "Living Room",
        4: "Bathroom"
    }
    
    new_room_name = room_mapping.get(new_room, None)
    current_room_name = room_mapping.get(current_room, None)
    
    if new_room_name and new_room_name != current_room_name:
        if current_room_name and lights[current_room_name] == 1:
            lights[current_room_name] = 0
            print(f"User left room {current_room_name}. Turning off the light.")
        
        if lights[new_room_name] == 0:
            lights[new_room_name] = 1
            print(f"User entered room {new_room_name}. Turning on the light.")
    return lights

# 控制灯光亮度
def adjust_light_brightness(room, brightness, lights_brightness):
    if brightness == 'increase':
        lights_brightness[room] = min(lights_brightness[room] + 1, 3)
        print(f"Brightness increased in room {room}. Current brightness level: {lights_brightness[room]}")
    elif brightness == 'decrease':
        lights_brightness[room] = max(lights_brightness[room] - 1, 0)
        print(f"Brightness decreased in room {room}. Current brightness level: {lights_brightness[room]}")

# 控制灯光开关
def control_light(state, room, lights, lights_brightness):
    if state == 'on':
        lights[room] = 1
        print(f"Turning on the light in room {room} via voice command.")
        lights_brightness[room] = 2
    elif state == 'off':
        lights[room] = 0
        print(f"Turning off the light in room {room} via voice command.")

# 控制窗帘
def control_curtain(state, room):
    if state == 'open':
        print(f"Opening the curtain in room {room} via voice command.")
    elif state == 'close':
        print(f"Closing the curtain in room {room} via voice command.")

# 控制空调
def control_ac(state, room):
    if state == 'on':
        print(f"Turning on the air conditioner in room {room} via voice command.")
    elif state == 'off':
        print(f"Turning off the air conditioner in room {room} via voice command.")

# 处理语音命令的功能
def process_command(command, lights, lights_brightness):
    if "turn on light" in command:
        if "bedroom" in command:
            control_light('on', "Bedroom", lights, lights_brightness)
        elif "kitchen" in command:
            control_light('on', "Kitchen", lights, lights_brightness)
    elif "turn off light" in command:
        if "bedroom" in command:
            control_light('off', "Bedroom", lights, lights_brightness)
        elif "kitchen" in command:
            control_light('off', "Kitchen", lights, lights_brightness)
    elif "brighten light" in command:
        if "bedroom" in command:
            adjust_light_brightness("Bedroom", 'increase', lights_brightness)
        elif "kitchen" in command:
            adjust_light_brightness("Kitchen", 'increase', lights_brightness)
    elif "dim light" in command:
        if "bedroom" in command:
            adjust_light_brightness("Bedroom", 'decrease', lights_brightness)
        elif "kitchen" in command:
            adjust_light_brightness("Kitchen", 'decrease', lights_brightness)
    elif "open curtain" in command:
        if "bedroom" in command:
            control_curtain('open', "Bedroom")
        elif "kitchen" in command:
            control_curtain('open', "Kitchen")
    elif "close curtain" in command:
        if "bedroom" in command:
            control_curtain('close', "Bedroom")
        elif "kitchen" in command:
            control_curtain('close', "Kitchen")
    elif "turn on aircondition" in command:
        if "bedroom" in command:
            control_ac('on', "Bedroom")
        elif "kitchen" in command:
            control_ac('on', "Kitchen")
    elif "turn off aircondition" in command:
        if "bedroom" in command:
            control_ac('off', "Bedroom")
        elif "kitchen" in command:
            control_ac('off', "Kitchen")
    else:
        print("Command not recognized or not supported.")

# 主函数：运行智能家居系统
def run_smart_home_system():
    svm_wifi_model, scaler = load_wifi_model()
    if svm_wifi_model is None or scaler is None:
        return

    current_room = None
    lights_brightness = {"Bedroom": 2, "Kitchen": 2, "Living Room": 2, "Bathroom": 2}
    lights = {"Bedroom": 0, "Kitchen": 0, "Living Room": 0, "Bathroom": 0}
    voice_command_active = False  # 标志位，表示是否有语音命令被执行

    while True:
        #  检查语音命令
        command = listen_to_user_command()
        if command:
            process_command(command, lights, lights_brightness)
            voice_command_active = True  # 语音命令被执行，暂停无线室内定位功能
            continue

        #  如果语音命令被执行，则暂停无线室内定位功能
        if voice_command_active:
            print("Voice command active. Pausing Wi-Fi localization until user leaves the room.")
            time.sleep(1)
            continue

        #  执行无线室内定位预测
        new_room = predict_room(svm_wifi_model, scaler)
        print(f"Predicted room: {new_room}, Current room: {current_room}")

        # 检测用户是否离开房间
        if new_room != current_room:
            lights = control_lights_based_on_room(new_room, current_room, lights)
            current_room = new_room
            voice_command_active = False  # 用户离开房间后，重置标志位，重新启用定位功能

        #  等待 1 秒，避免频繁循环
        time.sleep(1)


# 运行智能家居系统
if __name__ == "__main__":
    run_smart_home_system()




Models loaded successfully.
Listening for commands...
Sorry, I could not understand the command.
Predicted room: 3, Current room: None
User entered room Living Room. Turning on the light.
Listening for commands...
Sorry, I could not understand the command.
Predicted room: 3, Current room: 3
Listening for commands...
Sorry, I could not understand the command.
Predicted room: 4, Current room: 3
User left room Living Room. Turning off the light.
User entered room Bathroom. Turning on the light.
Listening for commands...
Sorry, I could not understand the command.
Predicted room: 1, Current room: 4
User left room Bathroom. Turning off the light.
User entered room Bedroom. Turning on the light.
Listening for commands...
Sorry, I could not understand the command.
Predicted room: 2, Current room: 1
User left room Bedroom. Turning off the light.
User entered room Kitchen. Turning on the light.
Listening for commands...
Sorry, I could not understand the command.
Predicted room: 3, Current room: 